In [28]:
import requests
import pandas as pd
import os
from pathlib import Path
import time
from datetime import datetime

def baixar_dados_ons():
    anos = list(range(2000, 2026))
    
    base_url = "https://ons-aws-prod-opendata.s3.amazonaws.com/dataset/ena_bacia_di/ENA_DIARIO_BACIAS_{ano}.csv"
    
    arquivos_baixados = []
    
    pasta_destino = Path("dados_ena_ons")
    pasta_destino.mkdir(exist_ok=True)
    
    print(" Iniciando download dos dados ENA do ONS...")
    print(f" Salvando em: {pasta_destino.absolute()}")
    
    for ano in anos:
        url = base_url.format(ano=ano)
        destino = pasta_destino / f"ENA_DIARIO_BACIAS_{ano}.csv"
        
        try:
            print(f" Baixando dados de {ano}...")
            
            response = requests.get(url, timeout=30)
            response.raise_for_status()
            
            with open(destino, "wb") as f:
                f.write(response.content)
            
            arquivos_baixados.append(destino)
            print(f" {ano} salvo em {destino}")
            
            time.sleep(0.5)
            
        except requests.exceptions.RequestException as e:
            print(f" Erro ao baixar {ano}: {e}")
        except Exception as e:
            print(f" Erro inesperado para {ano}: {e}")
    
    print(f"\n📊 Download concluído! {len(arquivos_baixados)} arquivos baixados.")
    return arquivos_baixados

In [29]:
def carregar_arquivos_ena(pasta_dados="dados_ena_ons"):
    pasta = Path(pasta_dados)
    
    if not pasta.exists():
        print(f" Pasta não encontrada: {pasta}")
        return None
    
    print("="*50)
    
    arquivos_csv = list(pasta.glob("ENA_DIARIO_BACIAS_*.csv"))
    
    if not arquivos_csv:
        print(" Nenhum arquivo ENA_DIARIO_BACIAS_*.csv encontrado")
        print(f" Procurando na pasta: {pasta.absolute()}")
        
        print(" Arquivos encontrados na pasta:")
        todos_arquivos = list(pasta.glob("*"))
        if todos_arquivos:
            for arquivo in todos_arquivos:
                print(f"   • {arquivo.name}")
        else:
            print("   (pasta vazia)")
        
        return None
    
    print(f"Encontrados {len(arquivos_csv)} arquivos na pasta: {pasta.absolute()}")
    print(f" Arquivos que serão carregados:")
    for arquivo in arquivos_csv:
        print(f"   • {arquivo.name}")
    
    df_list = []
    
    for arquivo in sorted(arquivos_csv):
        print(f"\n Carregando {arquivo.name}...")
        
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:  # ← ABRE O ARQUIVO
                primeira_linha = f.readline().strip()
            
            # Testa separadores comuns
            separadores = {';': 0, ',': 0, '\t': 0, '|': 0}
            for sep in separadores:
                separadores[sep] = primeira_linha.count(sep)
            
            # Escolhe o separador com mais ocorrências
            separador = max(separadores, key=separadores.get)
            num_colunas = separadores[separador] + 1
            
            print(f"   Separador: '{separador}' ({num_colunas} colunas esperadas)")
            
            df_temp = None
            encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
            
            for encoding in encodings:
                try:
                    df_temp = pd.read_csv(arquivo, sep=separador, encoding=encoding)  # ← LÊ O ARQUIVO
                    if len(df_temp.columns) > 1:
                        print(f"    Sucesso com encoding: {encoding}")
                        break
                except Exception:
                    continue
            
            # Se não funcionou, tenta detecção automática
            if df_temp is None or len(df_temp.columns) == 1:
                try:
                    df_temp = pd.read_csv(arquivo, sep=None, engine='python')  # ← SEGUNDA TENTATIVA
                    print(f"    Usando detecção automática de separador")
                except Exception as e:
                    print(f"    Falha total: {e}")
                    continue
            
            # Verifica se carregou corretamente
            if df_temp is not None and len(df_temp.columns) > 1:
                df_list.append(df_temp)  # ← ADICIONA À LISTA
                print(f"    Carregado: {len(df_temp):,} linhas x {len(df_temp.columns)} colunas")
                
                # Mostra algumas colunas para verificação
                colunas_sample = list(df_temp.columns[:5])
                if len(df_temp.columns) > 5:
                    colunas_sample.append("...")
                print(f"    Colunas: {colunas_sample}")
            else:
                print(f"    Arquivo não foi carregado corretamente")
                
        except Exception as e:
            print(f"    Erro inesperado: {e}")
    
    if not df_list:
        print("\n Nenhum arquivo foi carregado com sucesso")
        return None
    
    print(f"\n CONSOLIDANDO {len(df_list)} ARQUIVOS...")
    df_consolidado = pd.concat(df_list, ignore_index=True)  # ← JUNTA TUDO
    
    print(f" CONSOLIDAÇÃO CONCLUÍDA:")
    print(f"    Total: {len(df_consolidado):,} linhas")
    print(f"    Colunas: {len(df_consolidado.columns)}")
    print(f"    Lista de colunas:")
    for i, col in enumerate(df_consolidado.columns, 1):
        print(f"      {i:2d}. {col}")
    
    return df_consolidado


In [30]:
def analisar_dados(df):
    
    if df is None or df.empty:
        print(" DataFrame vazio para análise")
        return None
    
    print("="*50)
    
    total_linhas = len(df)
    total_colunas = len(df.columns)
    
    print(f" Dimensões: {total_linhas:,} linhas x {total_colunas} colunas")
    
    # Análise detalhada por coluna
    print(f"\n ANÁLISE POR COLUNA:")
    print(f"{'Coluna':<25} {'Tipo':<12} {'Nulos':<8} {'Vazios':<8} {'Total %':<8}")
    print("-" * 70)
    
    relatorio = {
        'total_linhas': total_linhas,
        'total_colunas': total_colunas,
        'colunas_problematicas': {},
        'colunas_ok': []
    }
    
    for col in df.columns:
        # Conta valores nulos
        nulos = df[col].isnull().sum()
        
        # Para colunas de texto, conta valores vazios
        vazios = 0
        if df[col].dtype == 'object':
            vazios = (df[col].astype(str).str.strip() == '').sum()
            vazios += (df[col] == '').sum()
        
        total_problemas = nulos + vazios
        percentual = (total_problemas / total_linhas) * 100
        
        # Exibe resultado
        col_nome = col[:23] + ".." if len(col) > 25 else col
        tipo = str(df[col].dtype)[:10]
        
        print(f"{col_nome:<25} {tipo:<12} {nulos:<8,} {vazios:<8,} {percentual:<7.1f}%")
        
        # Salva no relatório
        if total_problemas > 0:
            relatorio['colunas_problematicas'][col] = {
                'nulos': int(nulos),
                'vazios': int(vazios),
                'total_problemas': int(total_problemas),
                'percentual': round(percentual, 2),
                'tipo': str(df[col].dtype)
            }
        else:
            relatorio['colunas_ok'].append(col)
    
    # Análise de duplicatas
    duplicatas = df.duplicated().sum()
    linhas_vazias = df.isnull().all(axis=1).sum()
    
    print(f"\n RESUMO GERAL:")
    print(f"    Linhas duplicadas: {duplicatas:,}")
    print(f"    Linhas completamente vazias: {linhas_vazias:,}")
    print(f"    Colunas com problemas: {len(relatorio['colunas_problematicas'])}")
    print(f"    Colunas sem problemas: {len(relatorio['colunas_ok'])}")
    
    relatorio['duplicatas'] = int(duplicatas)
    relatorio['linhas_vazias'] = int(linhas_vazias)
    
    return relatorio

In [39]:
def limpar_dados(df, 
                           limite_nulos_coluna=0.7, 
                           remover_duplicatas=True, 
                           remover_vazias=True,
                           remover_linhas_com_nulos=True,
                           remover_linhas_com_zeros=True,
                           colunas_numericas_importantes=None):
    

    if df is None or df.empty:
        print(" DataFrame vazio para limpeza")
        return None
    
    print("="*50)
    
    df_limpo = df.copy()
    stats_inicial = {
        'linhas': len(df_limpo),
        'colunas': len(df_limpo.columns)
    }
    
    print(f" Dados iniciais: {stats_inicial['linhas']:,} linhas x {stats_inicial['colunas']} colunas")
        
    colunas_para_remover = []
    limite_absoluto = limite_nulos_coluna * len(df_limpo)
    
    for col in df_limpo.columns:
        nulos = df_limpo[col].isnull().sum()
        vazios = 0
        
        if df_limpo[col].dtype == 'object':
            vazios = (df_limpo[col].astype(str).str.strip() == '').sum()
            vazios += (df_limpo[col] == '').sum()
        
        total_problemas = nulos + vazios
        
        if total_problemas > limite_absoluto:
            percentual = (total_problemas / len(df_limpo)) * 100
            colunas_para_remover.append((col, percentual))
    
    if colunas_para_remover:
        print(f"    Removendo {len(colunas_para_remover)} colunas:")
        for col, perc in colunas_para_remover:
            print(f"      • {col} ({perc:.1f}% problemática)")
        
        colunas_nomes = [col for col, _ in colunas_para_remover]
        df_limpo = df_limpo.drop(columns=colunas_nomes)
    else:
        print("    Nenhuma coluna precisa ser removida")
    
    print("\n Limpando dados de texto...")
    
    colunas_texto = df_limpo.select_dtypes(include=['object']).columns
    for col in colunas_texto:
        df_limpo[col] = df_limpo[col].astype(str).str.strip()
        valores_vazios = ['', ' ', 'nan', 'NaN', 'null', 'NULL', 'None', 'none']
        df_limpo[col] = df_limpo[col].replace(valores_vazios, pd.NA)
    
    if len(colunas_texto) > 0:
        print(f"    {len(colunas_texto)} colunas de texto processadas")
    
    
    conversoes = 0
    for col in df_limpo.select_dtypes(include=['object']).columns:
        try:
            serie_numerica = pd.to_numeric(df_limpo[col], errors='coerce')
            valores_validos = serie_numerica.notna().sum()
            total_nao_nulos = df_limpo[col].notna().sum()
            
            if total_nao_nulos > 0 and (valores_validos / total_nao_nulos) > 0.8:
                df_limpo[col] = serie_numerica
                conversoes += 1
                print(f"   🔢 '{col}' convertida para numérica")
                
        except Exception:
            continue
    
    if conversoes == 0:
        print("    Nenhuma conversão de tipo necessária")
    
    if remover_linhas_com_nulos:
        linhas_com_nulos_antes = df_limpo.isnull().any(axis=1).sum()
        
        if linhas_com_nulos_antes > 0:
            df_limpo = df_limpo.dropna()
            linhas_removidas = linhas_com_nulos_antes
            print(f"    {linhas_removidas:,} linhas com nulos removidas")
        else:
            print("    Nenhuma linha com nulos encontrada")
    
    if remover_linhas_com_zeros:
        
        if colunas_numericas_importantes is None:
            colunas_numericas = df_limpo.select_dtypes(include=[int, float]).columns.tolist()
        else:
            colunas_numericas = [col for col in colunas_numericas_importantes if col in df_limpo.columns]
        
        if colunas_numericas:
            print(f"    Verificando zeros em colunas: {colunas_numericas}")
            
            # Conta linhas com pelo menos um zero nas colunas numéricas
            linhas_com_zeros_antes = (df_limpo[colunas_numericas] == 0).any(axis=1).sum()
            
            if linhas_com_zeros_antes > 0:
                df_limpo = df_limpo[~(df_limpo[colunas_numericas] == 0).any(axis=1)]
                linhas_removidas = linhas_com_zeros_antes
                print(f"    {linhas_removidas:,} linhas com zeros removidas")
            else:
                print("    Nenhuma linha com zeros encontrada")
        else:
            print("    Nenhuma coluna numérica encontrada para verificar zeros")
    
    if remover_vazias:
        linhas_vazias = df_limpo.isnull().all(axis=1).sum()
        
        if linhas_vazias > 0:
            df_limpo = df_limpo.dropna(how='all')
            print(f"    {linhas_vazias:,} linhas vazias removidas")
        else:
            print("    Nenhuma linha completamente vazia encontrada")
    
    if remover_duplicatas:
        print("\n Removendo duplicatas...")
        duplicatas = df_limpo.duplicated().sum()
        
        if duplicatas > 0:
            df_limpo = df_limpo.drop_duplicates()
            print(f"    {duplicatas:,} linhas duplicadas removidas")
        else:
            print("    Nenhuma duplicata encontrada")
    
    stats_final = {
        'linhas': len(df_limpo),
        'colunas': len(df_limpo.columns)
    }
    
    print(f"\n RESULTADO DA LIMPEZA AVANÇADA:")
    print(f"    Antes:  {stats_inicial['linhas']:,} linhas x {stats_inicial['colunas']} colunas")
    print(f"    Depois: {stats_final['linhas']:,} linhas x {stats_final['colunas']} colunas")
    
    linhas_removidas = stats_inicial['linhas'] - stats_final['linhas']
    colunas_removidas = stats_inicial['colunas'] - stats_final['colunas']
    
    if stats_inicial['linhas'] > 0:
        percentual_linhas = (linhas_removidas / stats_inicial['linhas']) * 100
        print(f"    Redução: {linhas_removidas:,} linhas ({percentual_linhas:.2f}%)")
    
    print(f"    Colunas removidas: {colunas_removidas}")
    
    # Verifica se ainda há dados
    if len(df_limpo) == 0:
        print("\n ATENÇÃO: Todos os dados foram removidos!")
        print("   Considere relaxar os critérios de limpeza.")
        return None
    
    return df_limpo


In [40]:
def salvar_dados(df, pasta_destino="dados_tratados_ena_ons", nome_arquivo="ENA_DADOS_TRATADOS"):
    
    if df is None or df.empty:
        print(" DataFrame vazio - não há dados para salvar")
        return None
    
    print("="*40)
    
    # Cria pasta se não existir
    pasta = Path(pasta_destino)
    pasta.mkdir(exist_ok=True)
    
    # Define nome do arquivo com timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    arquivo_csv = pasta / f"{nome_arquivo}_{timestamp}.csv"
    arquivo_principal = pasta / f"{nome_arquivo}.csv"
    
    try:
        # Salva arquivo principal (sobrescreve)
        df.to_csv(arquivo_principal, index=False, encoding='utf-8')
        print(f" Arquivo principal: {arquivo_principal}")
        
        # Salva arquivo com timestamp (backup)
        df.to_csv(arquivo_csv, index=False, encoding='utf-8')
        print(f" Arquivo com timestamp: {arquivo_csv}")
        
        # Cria arquivo de informações
        arquivo_info = pasta / f"{nome_arquivo}_INFO.txt"
        with open(arquivo_info, 'w', encoding='utf-8') as f:
            f.write("INFORMAÇÕES DOS DADOS TRATADOS\n")
            f.write("="*50 + "\n")
            f.write(f"Data de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"Arquivo principal: {arquivo_principal.name}\n")
            f.write(f"Arquivo backup: {arquivo_csv.name}\n\n")
            
            f.write("ESTATÍSTICAS:\n")
            f.write(f"- Total de linhas: {len(df):,}\n")
            f.write(f"- Total de colunas: {len(df.columns)}\n")
            f.write(f"- Tamanho do arquivo: {arquivo_principal.stat().st_size / 1024 / 1024:.2f} MB\n\n")
            
            f.write("COLUNAS DISPONÍVEIS:\n")
            for i, col in enumerate(df.columns, 1):
                tipo = str(df[col].dtype)
                nulos = df[col].isnull().sum()
                f.write(f"{i:2d}. {col} ({tipo}) - {nulos} nulos\n")
            
            f.write(f"\nPRIMEIRAS 5 LINHAS:\n")
            f.write(str(df.head().to_string()))
        
        print(f" Arquivo de informações: {arquivo_info}")
        
        # Mostra estatísticas finais
        print(f"\n ESTATÍSTICAS DO ARQUIVO SALVO:")
        print(f"    Linhas: {len(df):,}")
        print(f"    Colunas: {len(df.columns)}")
        print(f"    Tamanho: {arquivo_principal.stat().st_size / 1024 / 1024:.2f} MB")
        
        return str(arquivo_principal)
        
    except Exception as e:
        print(f" Erro ao salvar arquivos: {e}")
        return None

In [41]:
def tratar_dados_ena(pasta_dados="dados_ena_ons", 
                                    limite_nulos=0.7,
                                    remover_nulos=True,
                                    remover_zeros=True):
    print("="*60)
    
    df_original = carregar_arquivos_ena(pasta_dados)
    if df_original is None:
        return None
    
    relatorio = analisar_dados(df_original)
    
    colunas_ena_importantes = [
        'ena_bruta_bacia_mwmed',
        'ena_bruta_bacia_percentualmlt',
        'armazenavel_bacia_mwmed', 
        'armazenavel_bacia_percentualmlt'
    ]
    
    df_tratado = limpar_dados(
        df_original, 
        limite_nulos_coluna=limite_nulos,
        remover_duplicatas=True,
        remover_vazias=True,
        remover_linhas_com_nulos=remover_nulos,
        remover_linhas_com_zeros=remover_zeros,
        colunas_numericas_importantes=colunas_ena_importantes
    )
    
    if df_tratado is None:
        print("\n Nenhum dado restou após a limpeza!")
        return None
    
    # 5. Salvar resultado
    arquivo_salvo = salvar_dados(df_tratado, "dados_tratados_ena_ons", "ENA_DADOS_LIMPOS")
    
    if arquivo_salvo:
        print(f" Dados limpos salvos em: {arquivo_salvo}")
        
        
        print(f"\n ESTATÍSTICAS FINAIS:")
        print(f"   Linhas restantes: {len(df_tratado):,}")
        print(f"   Colunas restantes: {len(df_tratado.columns)}")
        
        nulos_restantes = df_tratado.isnull().sum().sum()
        zeros_restantes = (df_tratado.select_dtypes(include=[int, float]) == 0).sum().sum()
        
        print(f"   Valores nulos restantes: {nulos_restantes:,}")
        print(f"   Valores zeros restantes: {zeros_restantes:,}")
    
    return df_tratado

In [43]:
df_tratado = tratar_dados_ena(
    pasta_dados="dados_ena_ons",
    limite_nulos=0.7,
    remover_nulos=True,    # Remove linhas com qualquer nulo
    remover_zeros=True     # Remove linhas com zeros
)

Encontrados 26 arquivos na pasta: C:\Users\kevna\ml\dados_ena_ons
 Arquivos que serão carregados:
   • ENA_DIARIO_BACIAS_2000.csv
   • ENA_DIARIO_BACIAS_2001.csv
   • ENA_DIARIO_BACIAS_2002.csv
   • ENA_DIARIO_BACIAS_2003.csv
   • ENA_DIARIO_BACIAS_2004.csv
   • ENA_DIARIO_BACIAS_2005.csv
   • ENA_DIARIO_BACIAS_2006.csv
   • ENA_DIARIO_BACIAS_2007.csv
   • ENA_DIARIO_BACIAS_2008.csv
   • ENA_DIARIO_BACIAS_2009.csv
   • ENA_DIARIO_BACIAS_2010.csv
   • ENA_DIARIO_BACIAS_2011.csv
   • ENA_DIARIO_BACIAS_2012.csv
   • ENA_DIARIO_BACIAS_2013.csv
   • ENA_DIARIO_BACIAS_2014.csv
   • ENA_DIARIO_BACIAS_2015.csv
   • ENA_DIARIO_BACIAS_2016.csv
   • ENA_DIARIO_BACIAS_2017.csv
   • ENA_DIARIO_BACIAS_2018.csv
   • ENA_DIARIO_BACIAS_2019.csv
   • ENA_DIARIO_BACIAS_2020.csv
   • ENA_DIARIO_BACIAS_2021.csv
   • ENA_DIARIO_BACIAS_2022.csv
   • ENA_DIARIO_BACIAS_2023.csv
   • ENA_DIARIO_BACIAS_2024.csv
   • ENA_DIARIO_BACIAS_2025.csv

 Carregando ENA_DIARIO_BACIAS_2000.csv...
   Separador: ';' (6 coluna